In [ ]:
!pip install torch
!pip install transformers
!pip install kobert-transformers

In [ ]:
import pandas as pd
import re
import nltk
from kobert_transformers import get_tokenizer
from google.colab import drive

# NLTK stopwords 다운로드
nltk.download('stopwords')

# KoBERT 전용 토크나이저 초기화
tokenizer = get_tokenizer()

# Google Drive 마운트
drive.mount('/content/drive')

# 1. CSV 파일 불러오기
df = pd.read_csv('/content/drive/MyDrive/Team2/naver_news_comments.csv')

# 2. comment_id 열 삭제
df = df.drop(columns=['comment_id'])

# 3. comment_contents가 Null이거나 없거나 숫자만 있을 경우 해당 행 삭제
df = df[df['comment_contents'].notna()]  # Null 제거
df = df[df['comment_contents'].str.strip() != '']  # 빈 값 제거
df = df[~df['comment_contents'].str.isdigit()]  # 숫자만 있는 행 제거

# 4. 영어, 숫자, 특수기호, 한문 제거
df['comment_contents'] = df['comment_contents'].str.replace(r'[a-zA-Z]', '', regex=True)  # 영어 제거
df['comment_contents'] = df['comment_contents'].str.replace(r'\d+', '', regex=True)  # 숫자 제거
df['comment_contents'] = df['comment_contents'].str.replace(r'[^\w\s]', '', regex=True)  # 특수기호 제거
df['comment_contents'] = df['comment_contents'].apply(lambda x: re.sub(r'[\u4e00-\u9fff]', '', x))  # 한문 제거

# 5. 중복 데이터 제거
df = df.drop_duplicates(subset=['comment_contents'])

# 6. 앞뒤 공백 제거
df['comment_contents'] = df['comment_contents'].str.strip()

# 7. 'ㆍ'와 '"' 제거
df['comment_contents'] = df['comment_contents'].str.replace('ㆍ', '', regex=False)
df['comment_contents'] = df['comment_contents'].str.replace('"', '', regex=False)

# 8. 공백을 하나로 줄이기
df['comment_contents'] = df['comment_contents'].str.replace(r'\s+', ' ', regex=True)

# 9. KoBERT 토크나이저를 사용한 불용어 제거
def remove_stopwords(text):
    # KoBERT 토크나이저로 토큰화
    tokens = tokenizer.tokenize(text)
    # KoBERT 토큰 중 불필요한 것 제거 (특수 기호 제거 포함)
    filtered_tokens = [token for token in tokens if token.strip() and token != '▁']
    # 토큰을 다시 문자열로 결합
    return ' '.join(filtered_tokens)

df['comment_contents'] = df['comment_contents'].apply(remove_stopwords)

# 10. 남아 있는 KoBERT의 '▁' 기호 제거
df['comment_contents'] = df['comment_contents'].str.replace('▁', '', regex=False)

# 11. 공백을 하나로 줄이기 (다시 한번 처리)
df['comment_contents'] = df['comment_contents'].str.replace(r'\s+', ' ', regex=True)

# 12. comment_contents가 빈 값일 경우 해당 행 삭제
df = df[df['comment_contents'].str.strip() != '']

# 13. comment_date와 update_date에서 'T'와 그 이후의 숫자 제거
df['comment_date'] = df['comment_date'].str.replace(r'T\d+', '', regex=True)
df['update_date'] = df['update_date'].str.replace(r'T\d+', '', regex=True)

# 클렌징 완료 후 확인
print(df.head(10))

# 클렌징된 데이터 저장 (Google Drive의 'MyDrive' 폴더에 저장)
df.to_csv('/content/drive/MyDrive/Team2/naver_news_comments_cleaned.csv', index=False)
